In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-djf6qh0p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-djf6qh0p
  Resolved https://github.com/huggingface/transformers to commit 4005e30c804f9b3a9dbf45d019f6ca1cdc4d774c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 92.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install qwen_vl_utils av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 58.7 MB/s eta 0:00:00


In [7]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [4]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [37]:
# model = Qwen2VLForConditionalGeneration.from_pretrained(model_name, torch_type=torch.bfloat16, attn_implementation="flash_attention_2", device_map="auto"
# )
model = Qwen2VLForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
processor = AutoProcessor.from_pretrained(model_name)

In [50]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/content/qwennnn.mp4",
                "max_pixel": 360*420,
                "fps": 1.0,
            },
            {"type": "text", "text": "why did the doctor mark the knee area with hammer."},
        ],
    }
]

In [51]:
text = processor.apply_chat_template(
    messages, tokenize = False, add_generation_prompt = True
)

In [52]:
text

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>why did the doctor mark the knee area with hammer.<|im_end|>\n<|im_start|>assistant\n'

In [53]:
image_inputs, video_inputs = process_vision_info(messages)

In [54]:
inputs = processor(
    text = [text],
    images = image_inputs,
    videos = video_inputs,
    padding = True,
    return_tensors = "pt"
)

Unused or unrecognized kwargs: return_tensors.


In [55]:
inputs = inputs.to("cuda")

In [56]:
generated_ids = model.generate(**inputs, max_new_tokens = 512)

In [57]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

In [58]:
generated_ids_trimmed

[tensor([   785,  10668,    374,   1667,    264,  23457,    311,   1273,    279,
           8720,    594,  21381,  32666,    288,     13,   1096,    374,    264,
           4185,   1273,   1483,    311,   8552,    279,    729,    315,    279,
          21381,  10284,    323,    279,  49481,    304,    279,   2472,     13,
            576,  32666,    288,    525,  12510,    553,  52628,    279,  23457,
            389,    279,   8720,    594,  21381,    323,  44971,   3425,    279,
           8720,    594,  21381,  92797,    476,  10797,     13,   1096,   8609,
            279,  10668,   8253,    421,    279,  21381,    374,  30201,  10277,
            323,    421,   1052,    525,    894,   4714,    448,    279,  49481,
            476,  23648,    304,    279,   2472,     13, 151645],
        device='cuda:0')]

In [59]:
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens = True, clean_up_tokenization_spaces = False)

In [60]:
print(output_text)

["The doctor is using a hammer to test the patient's knee reflexes. This is a common test used to assess the function of the knee joint and the nerves in the leg. The reflexes are tested by tapping the hammer on the patient's knee and observing whether the patient's knee bends or moves. This helps the doctor determine if the knee is functioning properly and if there are any issues with the nerves or muscles in the leg."]
